# Masked language modeling

This file fine-tunes `roberta-large` using prompted-based method for depression detection.

In [1]:
!pip install transformers datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer, pipeline
import pandas as pd
from datasets import Dataset, DatasetDict
from tqdm.auto import tqdm
from sklearn.metrics import f1_score

try:
    from google.colab import drive
    drive.mount('/content/gdrive')

    train_path = '/content/gdrive/MyDrive/advanced-ml-project/data/train.tsv'
    test_path = '/content/gdrive/MyDrive/advanced-ml-project/data/test.tsv'
    dev_path = '/content/gdrive/MyDrive/advanced-ml-project/data/dev.tsv'

    eval_model_path = 'kwang123/MaskedLM-roberta-large'
except:
    train_path = 'data/train.tsv'
    test_path = 'data/test.tsv'
    dev_path = 'data/dev.tsv'

    eval_model_path = 'masked_model'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


device(type='cuda', index=0)

## Load Data

In [2]:
train = pd.read_csv(train_path, sep='\t', header=0)
train['label'] = train['label'].apply(lambda x: 'healthy' if x == 'not depression' else x)
train = train.sample(frac=1).reset_index(drop=True)

test = pd.read_csv(test_path, sep='\t', header=0)
test['label'] = test['label'].apply(lambda x: 'healthy' if x == 'not depression' else x)
test = test.sample(frac=1).reset_index(drop=True)

dev = pd.read_csv(dev_path, sep='\t', header=0)
dev['label'] = dev['label'].apply(lambda x: 'healthy' if x == 'not depression' else x)
dev = dev.sample(frac=1).reset_index(drop=True)
print(train.label.value_counts())
train.head(10)

moderate    6019
healthy     1971
severe       901
Name: label, dtype: int64


,PID,text,label
0,train_pid_4920,"I have no hope for this next decade, 2020 will...",moderate
1,train_pid_2115,What do you do when you feel unwanted? : For t...,moderate
2,train_pid_7098,Balancing reaching out and giving space? What’...,healthy
3,train_pid_4145,"I don't know how to deal with any of this, I n...",moderate
4,train_pid_599,Hope 2020 will be better for all of us : I wen...,moderate
5,train_pid_7170,Aye who else drunk rn im. Sad. Af and fighting...,healthy
6,train_pid_8706,Today i picked a date. : I made sure it was en...,severe
7,train_pid_1888,My depression poem it’s my first ever poem so ...,moderate
8,train_pid_4862,My tears mean nothing : [removed],moderate
9,train_pid_2983,just need someone to talk to : happy 2020 ever...,moderate


In [3]:
data = DatasetDict({
    'train': Dataset.from_pandas(train),
    'test': Dataset.from_pandas(test),
    'dev': Dataset.from_pandas(dev),
})
data

DatasetDict({
    train: Dataset({
        features: ['PID', 'text', 'label'],
        num_rows: 8891
    })
    test: Dataset({
        features: ['PID', 'text', 'label'],
        num_rows: 3245
    })
    dev: Dataset({
        features: ['PID', 'text', 'label'],
        num_rows: 4496
    })
})

## Preprocessing

In [4]:
tokenizer = AutoTokenizer.from_pretrained("roberta-large")
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    prompt = "The level of depression in the following tweet is {}.\n{}"
    texts = []
    for x, y in zip(examples["text"], examples["label"]):
        texts.append(prompt.format(y, x))
    return tokenizer(texts, padding='max_length', truncation=True, max_length=512)

tokenized_data = data.map(
    preprocess_function,
    batched=True,
    remove_columns=data["train"].column_names,
)

tokenized_data

Map:   0%|          | 0/8891 [00:00<?, ? examples/s]

Map:   0%|          | 0/3245 [00:00<?, ? examples/s]

Map:   0%|          | 0/4496 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 8891
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3245
    })
    dev: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4496
    })
})

In [5]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

## Training

In [ ]:
model = AutoModelForMaskedLM.from_pretrained("roberta-large")

training_args = TrainingArguments(
    output_dir="model_output",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    data_collator=data_collator,
)

trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.378700,0.454573
2,0.293600,0.441656
3,0.245500,0.432219


There were missing keys in the checkpoint model loaded: ['lm_head.decoder.weight', 'lm_head.decoder.bias'].


TrainOutput(global_step=3336, training_loss=0.3550902876636679, metrics={'train_runtime': 2536.0242, 'train_samples_per_second': 10.518, 'train_steps_per_second': 1.315, 'total_flos': 2.486152377692467e+16, 'train_loss': 0.3550902876636679, 'epoch': 3.0})

In [ ]:
trainer.save_model(eval_model_path)
tokenizer.save_pretrained(eval_model_path)

## Evaluation

In [6]:
from transformers import pipeline

In [7]:
mask_filler = pipeline(
    "fill-mask",
    eval_model_path,
    device=device,
    batch_size=16,
)

prompt = "The level of depression in this tweet is {}.\n{}"

predictions = mask_filler(
    [prompt.format(mask_filler.tokenizer.mask_token, x) for x in test['text'].tolist()],
    top_k=1,
    targets=['moderate', 'healthy', 'severe'],
    tokenizer_kwargs={'padding': 'max_length', 'truncation': True, 'max_length': 512}
)

In [8]:
label2idx = {'moderate': 0, 'healthy': 1, 'severe':2}

y_preds = [label2idx[each[0]['token_str'].strip()] for each in predictions]
y_true = [label2idx[each] for each in test['label'].tolist()]

weighted = f1_score(y_true, y_preds, average='weighted')
macro = f1_score(y_true, y_preds, average='macro')
print("Weighted F1 score: %.4f, Macro F1 score: %.4f" % (weighted, macro))

Weighted F1 score: 0.6160, Macro F1 score: 0.3786
